### when using dummyvec

In [ ]:
import numpy as np

T = 2048
N = N_ENVS

obs = venv.reset()

done_buf = np.zeros((T, N), dtype=np.float32)        # done flag from vecenv
trunc_buf = np.zeros((T, N), dtype=np.float32)       # time-limit truncation
bootstrap_mask = np.ones((T, N), dtype=np.float32)   # 1 means allow V(s_{t+1})

for t in range(T):
    actions = np.array([venv.action_space.sample() for _ in range(N)])
    next_obs, rewards, dones, infos = venv.step(actions)

    done_buf[t] = dones.astype(np.float32)

    # detect truncation
    trunc = np.array([info.get("TimeLimit.truncated", False) for info in infos], dtype=np.float32)
    trunc_buf[t] = trunc

    # if done and NOT trunc => true terminal => no bootstrap
    # if done and trunc => time-limit => DO bootstrap
    bootstrap_mask[t] = 1.0 - (done_buf[t] * (1.0 - trunc_buf[t]))

    obs = next_obs